In [709]:
import pandas as pd
import numpy as np
import math

In [710]:
traindf = pd.read_csv("train.csv")
traindf = traindf.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
traindf = traindf.fillna(0)
traindf["Fare"] /= 512.329
traindf["Age"] /= 80
traindf["SibSp"] /= 5
traindf["Parch"] /= 6
traindf["Pclass"] /= 3

In [711]:
traindf = traindf.to_numpy()
dataSize, paramSize = traindf.shape
paramSize -= 1

In [712]:
def encoder(data, col, size):
    b = {}
    cur = 0
    for i in range(size):
        if data[i][col] in b:
            data[i][col] = b[data[i][col]]
        else:
            b[data[i][col]] = cur
            data[i][col] = b[data[i][col]]
            cur += 1
    return data

def ReLU(Z, grad=False):
    if grad:
        return Z > 0
    return np.maximum(Z, 0)

def sigmoid(Z, grad=False):
    t = (1 / (1 + np.exp(-Z)))
    if grad:
        return t * (1- t)
    return t

def softmax(Z):
    A = np.exp(Z)+1 / sum(np.exp(Z)+1)
    return A

def get_predictions(A2):
    A2 = A2[0]
    for i in range(A2.shape[0]):
        if A2[i] > 0.8:
            A2[i] = 1
        else:
            A2[i] = 0
    return A2.reshape(A2.shape[0], 1).T

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def get_loss(pred, y):
    return sum(pred-y)


In [713]:
traindf = encoder(traindf, 2, dataSize)
traindf = encoder(traindf, 7, dataSize)
table = pd.DataFrame(traindf)

In [714]:
x = traindf.T[1:]
y = traindf.T[0]

In [715]:
def init_params(nin):
    w1 = np.random.rand(10, nin) 
    b1 = np.random.rand(10, 1)
    w2 = np.random.rand(1, 10)
    b2 = np.random.rand(1, 1)
    return w1, b1, w2, b2

def forward(w1, b1, w2, b2, x):
    a1 = np.dot(w1, x) + b1
    z1 = ReLU(a1)
    a2 = np.dot(w2, z1) + b2
    z2 = sigmoid(a2.astype(float))

    return a1, z1, a2, z2

def backward(w1, b1, w2, b2, a1, z1, a2, z2, x, y):
    loss = (z2 - y) * sigmoid(a2.astype(float), grad=True)
    dw2 = 1/dataSize * np.dot(loss, z1.T) 
    db2 = 1/dataSize * np.sum(loss)
    da1 = ReLU(a1, grad=True) * np.dot(w2.T, loss)
    dw1 = 1/dataSize * np.dot(da1, x.T)
    db1 = 1/dataSize * np.sum(da1)

    return dw1, db1, dw2, db2

def train(epochs, lr, x, y):
    w1, b1, w2, b2 = init_params(paramSize)
    for epoch in range(epochs+1):
        a1, z1, a2, z2 = forward(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = backward(w1, b1, w2, b2, a1, z1, a2, z2, x, y)
        w1 = w1 - dw1 * lr
        b1 = b1 - db1 * lr
        w2 = w2 - dw2 * lr
        b2 = b2 - db2 * lr
        if (epoch%50 == 0):
            norm = get_predictions(z2)
            loss = get_loss(norm, y)
            acc = get_accuracy(norm, y)
            
            print(f"Epoch: {epoch}   Accuracy: {acc}")
    return w1, b1, w2, b2



In [718]:
w1, b1, w2, b2 = train(50000, 10, x, y)

Epoch: 0   Accuracy: 0.3838383838383838
Epoch: 50   Accuracy: 0.7968574635241302
Epoch: 100   Accuracy: 0.8013468013468014
Epoch: 150   Accuracy: 0.8047138047138047
Epoch: 200   Accuracy: 0.8047138047138047
Epoch: 250   Accuracy: 0.8058361391694725
Epoch: 300   Accuracy: 0.8069584736251403
Epoch: 350   Accuracy: 0.8080808080808081
Epoch: 400   Accuracy: 0.8080808080808081
Epoch: 450   Accuracy: 0.8103254769921436
Epoch: 500   Accuracy: 0.8114478114478114
Epoch: 550   Accuracy: 0.8125701459034792
Epoch: 600   Accuracy: 0.8159371492704826
Epoch: 650   Accuracy: 0.8159371492704826
Epoch: 700   Accuracy: 0.8159371492704826


KeyboardInterrupt: 

In [ ]:
def over_train(w1, b1, w2, b2, epochs, lr, x, y):
    for epoch in range(epochs+1):
        a1, z1, a2, z2 = forward(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = backward(w1, b1, w2, b2, a1, z1, a2, z2, x, y)
        w1 = w1 - dw1 * lr
        b1 = b1 - db1 * lr
        w2 = w2 - dw2 * lr
        b2 = b2 - db2 * lr
        if (epoch%50 == 0):
            norm = get_predictions(z2)
            loss = get_loss(norm, y)
            acc = get_accuracy(norm, y)
            
            
            print(f"Epoch: {epoch}   Accuracy: {acc}")
    return w1, b1, w2, b2

In [ ]:
w1, b1, w2, b2 = over_train(w1, b1, w2, b2, 5000, 10, x, y)

Epoch: 0   Accuracy: 0.6734006734006734
Epoch: 50   Accuracy: 0.8047138047138047
Epoch: 100   Accuracy: 0.7991021324354658
Epoch: 150   Accuracy: 0.8002244668911336
Epoch: 200   Accuracy: 0.8013468013468014
Epoch: 250   Accuracy: 0.8013468013468014
Epoch: 300   Accuracy: 0.8013468013468014
Epoch: 350   Accuracy: 0.8024691358024691
Epoch: 400   Accuracy: 0.8035914702581369
Epoch: 450   Accuracy: 0.8047138047138047
Epoch: 500   Accuracy: 0.8069584736251403
Epoch: 550   Accuracy: 0.8080808080808081
Epoch: 600   Accuracy: 0.8092031425364759
Epoch: 650   Accuracy: 0.8103254769921436
Epoch: 700   Accuracy: 0.8080808080808081
Epoch: 750   Accuracy: 0.8080808080808081
Epoch: 800   Accuracy: 0.8080808080808081
Epoch: 850   Accuracy: 0.8080808080808081
Epoch: 900   Accuracy: 0.8047138047138047
Epoch: 950   Accuracy: 0.8069584736251403
Epoch: 1000   Accuracy: 0.8069584736251403
Epoch: 1050   Accuracy: 0.8058361391694725
Epoch: 1100   Accuracy: 0.8069584736251403
Epoch: 1150   Accuracy: 0.80920314

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
test = test.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
test = test.fillna(1)

In [ ]:

test["Fare"] /= 512.329
test["Age"] /= 80
test["SibSp"] /= 5
test["Parch"] /= 6
test["Pclass"] /= 3
test = test.to_numpy()
dataSize, paramSize = test.shape
test = encoder(test, 1, dataSize)
test = encoder(test, 6, dataSize)


In [ ]:
a1, z1, a2, z2 = forward(w1, b1, w2, b2, test.T)
test_pred = get_predictions(z2)

In [ ]:
predictions = pd.DataFrame(test_pred.T)
predictions = pd.DataFrame({'PassengerId': range(892, len(predictions) + 892), 'Survived': predictions[0].astype(int)})
predictions.to_csv("submission.csv", index=False)
